In [1]:
# Let's try with sympy

import numpy as np
import sympy

embedding_dim = 2
hidden_dim = 3

def random_matr(*shape):
    return sympy.Matrix(np.random.normal(size=shape).round(1))

A     = random_matr(hidden_dim, 2 * embedding_dim)
A_hat = random_matr(*A.shape)
b     = random_matr(hidden_dim, 1)
b_hat = random_matr(*b.shape)
C     = random_matr(hidden_dim, hidden_dim)
C_hat = random_matr(*C.shape)
d     = random_matr(hidden_dim, 1)
d_hat = random_matr(*d.shape)
E     = random_matr(1, hidden_dim)
E_hat = random_matr(*E.shape)
f     = random_matr(1, 1)
f_hat = random_matr(*f.shape)

params = sympy.symbols(["β"])
beta = params[0]

class relu(sympy.Function):
    @classmethod
    def eval(cls, x):
        return x.applyfunc(lambda elem: sympy.Max(elem, 0))

    def _eval_is_real(self):
        return True

delta_e1 = np.random.normal(size=(2 * embedding_dim, 1)).round(1)
delta_e2 = np.random.normal(size=(2 * embedding_dim, 1)).round(1)
# the first half of the vectors is always the same
delta_e2[:embedding_dim, :] = delta_e1[:embedding_dim, :]
delta_e1 = sympy.Matrix(delta_e1)
delta_e2 = sympy.Matrix(delta_e2)

def q(beta, x):
    result = relu((A + beta * A_hat) @ x + b + beta * b_hat)
    result = relu((C + beta * C_hat) @ result + d + beta * d_hat)
    result = (E + beta * E_hat) @ result + f + beta * f_hat
    return result

def to_scalar(x):
    assert x.shape == (1, 1)
    return x[0, 0]
    
logit = to_scalar(q(beta, delta_e1) - q(beta, delta_e2))
print(logit)
dlogit_dbeta = logit.diff(beta)
print(dlogit_dbeta)

-(1.1 - 0.8*β)*Max(0, -0.8*β + (0.1 - 0.8*β)*Max(0, 2.19*β - 0.49) + (0.2 - 0.2*β)*Max(0, 0.44 - 0.37*β) + (0.1*β - 1.2)*Max(0, -0.44*β - 1.02) - 0.6) + (1.1 - 0.8*β)*Max(0, -0.8*β + (0.1 - 0.8*β)*Max(0, 3.21*β + 0.41) + (0.2 - 0.2*β)*Max(0, 1.16 - 0.07*β) + (0.1*β - 1.2)*Max(0, -1.28*β - 1.5) - 0.6) - (0.3*β + 0.1)*Max(0, -1.7*β + (0.2 - 0.5*β)*Max(0, 2.19*β - 0.49) + (0.6 - 1.0*β)*Max(0, 0.44 - 0.37*β) + (0.6 - 0.4*β)*Max(0, -0.44*β - 1.02) + 0.5) + (0.3*β + 0.1)*Max(0, -1.7*β + (0.2 - 0.5*β)*Max(0, 3.21*β + 0.41) + (0.6 - 1.0*β)*Max(0, 1.16 - 0.07*β) + (0.6 - 0.4*β)*Max(0, -1.28*β - 1.5) + 0.5) - (0.4*β + 0.5)*Max(0, -0.3*β + (1.1 - 1.2*β)*Max(0, 2.19*β - 0.49) + (-1.4*β - 1.5)*Max(0, -0.44*β - 1.02) + (-0.4*β - 1.4)*Max(0, 0.44 - 0.37*β) - 1.9) + (0.4*β + 0.5)*Max(0, -0.3*β + (1.1 - 1.2*β)*Max(0, 3.21*β + 0.41) + (-1.4*β - 1.5)*Max(0, -1.28*β - 1.5) + (-0.4*β - 1.4)*Max(0, 1.16 - 0.07*β) - 1.9)
(1.1 - 0.8*β)*(3.21*(0.1 - 0.8*β)*Heaviside(3.21*β + 0.41) - 0.07*(0.2 - 0.2*β)*Heavisid

In [2]:
def estimate_upper_bound(expr, param_name, abs_param_bound):
    if type(expr).__name__ == "Add":
        arg_bounds = [estimate_upper_bound(x, param_name, abs_param_bound) for x in expr.args]
        return sum(arg_bounds)
    if type(expr).__name__ == "Mul":
        arg_bounds = [estimate_upper_bound(x, param_name, abs_param_bound) for x in expr.args]
        return np.prod(arg_bounds)
    if type(expr).__name__ == "Float":
        return np.abs(float(expr))
    if type(expr).__name__ == "NegativeOne":
        return 1.0
    if type(expr).__name__ == "Symbol":
        if str(expr) == param_name:
            return abs_param_bound
        else:
            raise RuntimeError(f"Unexpected symbol {expr}")
    if type(expr).__name__ == "Max":
        if len(expr.args) == 2 and str(expr.args[0] == "0"):
            return estimate_upper_bound(expr.args[1], param_name, abs_param_bound)
        else:
            raise RuntimeError(f"Unexpected Max expression {expr}")
    if type(expr).__name__ == "Heaviside":
        return 1.0
    else:
        raise RuntimeError(f"Unexpected type {type(expr)}")

print(estimate_upper_bound(logit, "β", 10))
print(estimate_upper_bound(dlogit_dbeta, "β", 10))

poly_params = [sympy.Function(f"p{i}") for i in range(3)]
p0 = poly_params[0]
p1 = poly_params[1]
p2 = poly_params[2]
poly = 1 - p0(beta)*p1(beta) + 2*p1(beta)*p2(beta)
dpoly_dbeta = poly.diff(beta)
print(dpoly_dbeta)

# TODO
#  compute a pool of bounds
#  construct kappa
#  substitute ps with functions
#  get symbolic derivative
#  recursive estimation for the symbolic derivative
#    see Derivative(p_i) -> subsitute the bound
#    see just p_i -> subsitute with the bound

# if necessary, take the second derivative?

10991.693599999999
3016.3206
-p0(β)*Derivative(p1(β), β) - p1(β)*Derivative(p0(β), β) + 2*p1(β)*Derivative(p2(β), β) + 2*p2(β)*Derivative(p1(β), β)


In [3]:
## Let's upper- and lower- bound the derivative with a solver

class sigmoid(sympy.Function):
    @classmethod
    def eval(cls, x):
        return (sympy.functions.elementary.hyperbolic.tanh(2 * x) + 1) / 2

    def _eval_is_real(self):
        return True

smoothing_alpha = 0.01
    
full_poly = 1 - ((1 - smoothing_alpha)*sigmoid(logit) + smoothing_alpha/2)
#print(full_poly)

In [160]:
#solutions = sympy.solve(full_poly - 1, beta)
#print(solutions)
# too long even for small dimension

In [13]:
# see what the expressions become without ReLU and Heaviside

def simplify(expr):
    if type(expr).__name__ == "Add":
        return sum([simplify(x) for x in expr.args])
    if type(expr).__name__ == "Mul":
        result = expr.args[0]
        for x in expr.args:
            result *= simplify(x)
        return result
    if type(expr).__name__ in ["Float", "NegativeOne", "Symbol"]:
        return expr
        if str(expr) == param_name:
            return param
        else:
            raise RuntimeError(f"Unexpected symbol {expr}")
    if type(expr).__name__ == "Max":
        if len(expr.args) == 2 and str(expr.args[0] == "0"):
            return simplify(expr.args[1])
        else:
            raise RuntimeError(f"Unexpected Max expression {expr}")
    if type(expr).__name__ == "Heaviside":
        return simplify(expr.args[0])
    else:
        raise RuntimeError(f"Unexpected type {type(expr)}")

print(simplify(dlogit_dbeta).simplify())


-42.8275722929638*β**7 - 978.863970222645*β**6 - 1934.77145235517*β**5 + 363.741698379608*β**4 + 1842.7998904518*β**3 + 271.143807826922*β**2 - 145.179926934488*β - 9.17297400131761
